# Predicting Heart Failure
## Machine Learning with Logistic Regression

### Import Modules

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import seaborn as sn
plt.style.use('./deeplearning.mplstyle')